In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Hypothyroidism/GSE32445'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identical gene regulation patterns of triiodothyronine (T3) and selective thyroid hormone receptor modulator GC-1"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell line: HepG2'], 1: ['cell type: hepatoma cells']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is likely available
is_gene_available = True  # Based on the series title mentioning gene regulation

# Analyzing Sample Characteristics Dictionary
# Variable 'Hypothyroidism': Not Available
trait_row = None

# Variable 'age': Available and located at key 2
age_row = 2

# Variable 'gender': Available and located at key 1
gender_row = 1

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    gender = value.split(': ')[1].lower()
    if gender == 'male':
        return 1
    elif gender == 'female':
        return 0
    return None

save_cohort_info('GSE32445', './preprocessed/Hypothyroidism/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
# Since trait_row is None, skip this step.
